## Baseball Machine Learning - Classification
### Data Prep and Cleaning

This notebook contains my work on cleaning and preparing a dataset to be used for several classification algorithms in a future project. Starting with a set of csv data files obtained from the Baseball Databank (acq. from http://www.seanlahman.com/baseball-archive/statistics/), I used a variety of Pandas methods to prepare the data into a single, aggregated data source containing a collection of relevant variables. 
In a previous iteration of this project, I used several machine learning classification algorithms to classify and predict whether baseball players would be inducted into the Hall of Fame. That project utilized 5 dependent variables, all related to players' hitting performance; this project's dataset contains over 20 features covering the players' hitting and fielding, as well as their All Star and award nominations. Ideally, this more robust dataset will lead to more accurate and statistically significant results when the classification algorithms are applied in future steps.

The notebook is broken up into seperate sections relating to these feature groups, starting with Fielding.

In [1]:
import pandas as pd

batting = pd.read_csv('Batting.csv')
hof = pd.read_csv('HallOfFame.csv')

hof = hof.groupby('playerID').yearid.max().reset_index() #note this removes the binary induction column
#hof.head()
#print(len(hof)) = 1239

## Fielding
Starting with fielding stats, I imported the dataset containing players' season-by-season fielding metrics, _fielding.csv_ , and merged it with the Hall of Fame dataset to isolate only players who appeared on a HoF ballot. Initially I had wanted to track Zone Rating as a coverall metric for fielding aptitude, but a cursory look into the data revealed that ZR data was only available for 41 players, so the metric was abandoned in favor of others. Other data cleaning actions taken were the removal of pitchers (due to differences in statistics, this project will only look at hitters), the selection of relevant performance metrics, and filling empty data with 0's to allow for aggregation and stat calculation.

In [2]:
fielding = pd.read_csv('Fielding.csv')
hof_fielding = pd.merge(hof, fielding, on='playerID')

#remove pitchers:
hof_fielding = hof_fielding[hof_fielding.POS != 'P']

ZR_fielding = hof_fielding[hof_fielding['ZR'].notnull()]
ZR_fielding = ZR_fielding.groupby(['playerID', 'yearid']).ZR.sum().reset_index()

#for some reason, ZR stats are only tracked for 41 distinct players, ranging from the 1960's to present

fielding_data = hof_fielding[['playerID', 'G', 'PO', 'E', 'A']]
fielding_data = fielding_data.fillna(0)
fielding_data.head()

,playerID,G,PO,E,A
0,aaronha01,116,223,7.0,5
1,aaronha01,27,86,6.0,84
2,aaronha01,126,254,9.0,9
3,aaronha01,152,316,13.0,17
4,aaronha01,150,346,6.0,9


A major problem with the data I imported was that it contained only season-by-season stats, while this project is concerned with overall career stats. Therefore, I needed to find a way to group the data by player, and sum it all up to create career-length aggregate stats. In a previous iteration of this project, I did this by creating a function to iterate over the data and sum it, but this method was very computationally expensive, so in the following cells I tried 2 different methods to aggregate the data. Both methods seem to do the job, however a preliminary test (demonstrated below) reveals that method 2 is much faster. As such it will primarily be used for the rest of this notebook, with the exception of the batting dataframe.

In [3]:
%timeit method1 = fielding_data.groupby('playerID')[['G', 'PO', 'E', 'A']].apply(lambda x: x.astype(int).sum()).reset_index()

1.36 s ± 177 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit method2 = fielding_data.groupby('playerID').agg({'G':'sum', 'PO':'sum', 'E':'sum', 'A':'sum'}).reset_index()

11.6 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
#method 1 
fielding_agg = fielding_data.groupby('playerID').agg({'G':'sum', 'PO':'sum', 'E':'sum', 'A':'sum'}).reset_index()
fielding_agg = fielding_agg[fielding_agg.G >= 100]
fielding_agg.head()

,playerID,G,PO,E,A
0,aaronha01,3020,7436,144.0,429
1,adamsbo03,1066,1748,158.0,2262
2,adamssp01,1384,2287,230.0,3748
3,ageeto01,1073,2371,61.0,53
5,allenbe01,1023,2031,100.0,2347


Due to the limitations of the fielding data, I wasn't able to use any advanced fielding metrics (ie UZR, Total Zone), so I settled for Range Factor and Fielding Percentage as general metrics to hopefully estimate players' fielding ability. Both Fielding Percentage and Total Zone could be calculated using the variables I had available, which was done with the functions defined below. Overall, the fielding metrics which will be tracked are __PO__ (putouts), __E__ (errors), __RF__ (range factor), and __FP__ (fielding percentage). While not comprehensive, these will hopefully cover players' fielding ability to an acceptable degree.

Assists (__A__) were necessary for the calculations but were later dropped as they carry little weight and would likely be biased towards infielders and catchers.

In [6]:
#range factor:
def range_factor(data):
    return((data['A']+data['PO'])/(data['G']))

fielding_agg['RF'] = fielding_agg.apply(lambda row: range_factor(row), axis=1)
fielding_agg['RF'] = round(fielding_agg['RF'], 2)

#fielding percentage:
def fp(data):
    APO = data['A'] + data['PO']
    return round(APO/(APO + data['E']), 3)

fielding_agg['FP'] = fielding_agg.apply(lambda row: fp(row), axis=1)
fielding_agg.head()
#print(len(fielding_agg)) = 1174
#print(fielding_agg.playerID.nunique()) = 1174

,playerID,G,PO,E,A,RF,FP
0,aaronha01,3020,7436,144.0,429,2.60,0.982
1,adamsbo03,1066,1748,158.0,2262,3.76,0.962
2,adamssp01,1384,2287,230.0,3748,4.36,0.963
3,ageeto01,1073,2371,61.0,53,2.26,0.975
5,allenbe01,1023,2031,100.0,2347,4.28,0.978


## Batting
In my previous iteration of this project idea, I tracked only 5 features for each player, all of which focused on hitting (G, AB, H, HR, BA). In this version, along with features from other categories (fielding, awards), I have also added more batting variables. Primarily, this includes calculations of rate stats (on base percentage, slugging percentage, and on-base-plus-slugging), which will hopefully encompass much more variation in players' skills and performance. I also added doubles, triples, RBI's, walks, and strikeouts, as well as stolen bases to cover players' baserunning ability. 
At this point the Batting dataframe still contains stats for pitchers who hit at some point in their careers. Unlike the Fielding dataframe, the batting data doesn't have a position column so it was impossible to isolate pitchers. Therefore, they appear in these calculations (ie playerID 'abbotji01' on row 1 below is pitcher Jim Abbott), though they will be removed later when the dataframes are merged. 

In [7]:
batting_hof = pd.merge(hof, batting, on='playerID')
batting_hof = batting_hof.fillna(0)

batting_agg = batting_hof.groupby('playerID')[['AB', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'SO', 'BB', 'HBP']].apply(lambda x: x.astype(int).sum()).reset_index()
#print(len(batting_hof))
#list(batting_hof.columns)
batting_agg.head()

,playerID,AB,H,2B,3B,HR,RBI,SB,SO,BB,HBP
0,aaronha01,12364,3771,624,98,755,2297,240,1383,1402,32
1,abbotji01,21,2,0,0,0,3,0,10,0,0
2,adamsba01,1019,216,31,15,3,75,1,194,53,1
3,adamsbo03,4019,1082,188,49,37,303,67,447,414,17
4,adamssp01,5557,1588,249,48,9,394,154,223,453,28


A series of simple aggregate statements to calculate traditional baseball 'rate stats': OBP, SLG, and OPS. There will likely be some collinearity between these variables (OPS is just OBP+SLG), but I felt it was important to include all 3 since OBP and SLG convey important information about players' hitting tendencies (ie how much they walk, whether they tend to hit for power) which could be missed by only using batting average and OPS. Several variables (TB, AB, HBP) were necessary for the calculation of these rate stats but contain little useful information on their own, so they were dropped from the dataframe after the rate stats were added.

In [8]:
#rate stats:
batting_agg['BA'] = round(batting_agg['H']/batting_agg['AB'], 3)
batting_agg['TB'] = (batting_agg['H']-batting_agg['2B']-batting_agg['3B']-batting_agg['HR'])+(2*batting_agg['2B'])+(3*batting_agg['3B'])+(4*batting_agg['HR'])
batting_agg['SLG'] = round(batting_agg['TB']/batting_agg['AB'], 3)
batting_agg['OBP'] = round((batting_agg['H']+batting_agg['BB']+batting_agg['HBP'])/batting_agg['AB'], 3)
batting_agg['OPS'] = batting_agg['OBP']+batting_agg['SLG']

In [9]:
#remove extraneous columns:
batting_agg_2 = batting_agg.drop(['AB', 'TB', 'HBP'], axis=1)
batting_agg_2.head()

,playerID,H,2B,3B,HR,RBI,SB,SO,BB,BA,SLG,OBP,OPS
0,aaronha01,3771,624,98,755,2297,240,1383,1402,0.305,0.555,0.421,0.976
1,abbotji01,2,0,0,0,3,0,10,0,0.095,0.095,0.095,0.190
2,adamsba01,216,31,15,3,75,1,194,53,0.212,0.281,0.265,0.546
3,adamsbo03,1082,188,49,37,303,67,447,414,0.269,0.368,0.376,0.744
4,adamssp01,1588,249,48,9,394,154,223,453,0.286,0.353,0.372,0.725


## Awards
A more subjective category of features which I wanted to add in the second iteration of this project is award and all star accumulation. Hall of Fame voting is not purely a numeric process but often relies on voters' subjective opinions of players and their HoF 'worthiness'. This is difficult to quantify in a model, though I hope that including players' career accolades might at least cover some of this subjective dimension of the induction process.

This step began with aggregating players' all star game appearances, which was simple enough using an aggregate command attached to a groupby statement. The data was obtained from the _AllstarFull_ csv file in the Baseball Databank. Note that there are only 719 unique player rows in the allstar dataframe, compared to the final dataframe, which contains 818 players - there have been players who appeared on a Hall of Fame ballot despite not playing in an allstar game. This will be resolved later by merging the Allstar dataframe to the rest of the data via an outer join, and replacing the 'NaN' datapoints with 0's to reflect those players' 0 allstar appearances. 

In [10]:
allstar_full = pd.read_csv('AllstarFull.csv')
allstar_hof = pd.merge(hof, allstar_full, on='playerID')

#allstar_hof.head()
allstar = allstar_hof.groupby('playerID').agg({'yearID':'count'}).reset_index()
allstar.rename(columns={'yearID':'ASGs'}, inplace=True)
allstar.head()
#print(len(allstar)) = 719

,playerID,ASGs
0,aaronha01,24
1,ageeto01,2
2,aguilri01,3
3,alexado01,1
4,allendi01,7


The next 2 cells illustrate the process of adding the award features to the dataset. The awards that I chose to add as features are MVP, Gold Glove (which hopefully reflects fielding ability as well), and Rookie of the Year. These awards were chosen in part because there is sufficient data for them; these are 3 of the oldest continuous awards in baseball (the 'newest' is the GG, introduced in the 1950's).

For this process I used the _AwardsPlayers_ csv from the Baseball Databank, which listed winners of the award by year. I first isolated the awards of interest, then merged the awards df with the hall of fame df to index only players who appeared on a HoF ballot. Finally, I used a series of lambda functions to convert the strings in the __awardID__ column into seperate binary columns, where a 1 would indicate whether the player won a specific award in a certain year. Finally, I aggregated these binary values to produce a dataframe with total counts for each award for each player across their whole career. 

In [11]:
awards = pd.read_csv('AwardsPlayers.csv')
#print(awards.awardID.unique())

#select awards of interest:
awards = awards[(awards.awardID == 'Most Valuable Player')| (awards.awardID == 'Rookie of the Year')|(awards.awardID == 'Gold Glove')]
awards = awards[['playerID', 'awardID']]
#awards.playerID.nunique() = 514

awards.head()

,playerID,awardID
188,cobbty01,Most Valuable Player
189,schulfr01,Most Valuable Player
226,speaktr01,Most Valuable Player
227,doylela01,Most Valuable Player
258,johnswa01,Most Valuable Player


In [12]:
hof_awards = pd.merge(hof, awards, on='playerID')
#hof_awards.playerID.nunique() = 317

#create dummy 
hof_awards['MVP'] = hof_awards.apply(lambda row: 1 if row['awardID'] == 'Most Valuable Player' else 0, axis=1)
hof_awards['GG'] = hof_awards.apply(lambda row: 1 if row['awardID'] == 'Gold Glove' else 0, axis=1)
hof_awards['RoY'] = hof_awards.apply(lambda row: 1 if row['awardID'] == 'Rookie of the Year' else 0, axis=1)

#isolate dummy variable columns:
hof_awards = hof_awards[['playerID', 'MVP', 'GG', 'RoY']]
hof_awards = hof_awards.groupby('playerID').agg({'MVP':'sum', 'GG':'sum', 'RoY':'sum'}).reset_index()

hof_awards.head()
#print(len(hof_awards)) = 287

,playerID,MVP,GG,RoY
0,aaronha01,1,3,0
1,ageeto01,0,2,1
2,allendi01,1,0,1
3,alleyge01,0,2,0
4,alomaro01,0,10,0


## Hall of Fame Status
The final step of the data prep stage is to add a binary variable for Hall of Fame induction status, which will become the variable of interest for the model (1 if the player is inducted into the Hall, 0 otherwise). This feature was added simply enough using the _HallofFame_ csv file, by converting the string value (Y/N) of the induction column into a binary 1/0. The data is then grouped by __playerID__ and the max value of __inducted_bin__ is selected; this will leave us with a single datapoint for each player (many players appeared on multiple ballots before being inducted/rejected, so this step filters out the extraneous data). 

In [13]:
hof_2 = pd.read_csv('HallOfFame.csv')

hof_2['inducted_bin'] = hof_2['inducted'].apply(lambda x: 1 if x == 'Y' else 0)

hof_2 = hof_2.groupby('playerID').inducted_bin.max().reset_index()

hof_2.head()
#print(len(hof_2)) = 1239

,playerID,inducted_bin
0,aaronha01,1
1,abbotji01,0
2,adamsba01,0
3,adamsbo03,0
4,adamssp01,0


## Mergers
At this point in the process we are left with several dataframes containing different features, of various lengths. The upcoming model will utilize a single data set for simplicity, which will be created by merging all of the previous dataframes together. This starts by merging the __Batting__ and __Fielding__ data; because pitchers were dropped from the fielding data earlier, this inner merge will drop the pitchers from the batting data also, leaving us with just the batting and fielding data for hitters. 

In [14]:
bf_data = pd.merge(fielding_agg, batting_agg_2, on='playerID')
#remove assists: likely biased towards inf/catchers
bf_data = bf_data.drop(columns=['A'])
bf_data.head()

,playerID,G,PO,E,RF,FP,H,2B,3B,HR,RBI,SB,SO,BB,BA,SLG,OBP,OPS
0,aaronha01,3020,7436,144.0,2.60,0.982,3771,624,98,755,2297,240,1383,1402,0.305,0.555,0.421,0.976
1,adamsbo03,1066,1748,158.0,3.76,0.962,1082,188,49,37,303,67,447,414,0.269,0.368,0.376,0.744
2,adamssp01,1384,2287,230.0,4.36,0.963,1588,249,48,9,394,154,223,453,0.286,0.353,0.372,0.725
3,ageeto01,1073,2371,61.0,2.26,0.975,999,170,27,130,433,167,918,342,0.255,0.412,0.351,0.763
4,allenbe01,1023,2031,100.0,4.28,0.978,815,140,21,73,352,13,424,370,0.239,0.357,0.350,0.707


Next the awards and allstar data is added; since there were players for whom we had batting/fielding data but no awards data (either because they didn't win any awards or because they played before certain awards were introduced), an outer join is used here to include these players. By default the outer join will list any empty datapoints as __NA__ , but these are converted to 0's for future regularization.

The missing data in the awards section may end up becoming a problem when using this data for classification and modeling; players who played before the introduction of these awards may have some unfair bias leveled against them. I suspect that because of this the awards data may not contribute much to the model; however at this point I chose to leave it in, as it can be removed later if its contributions to the model are negligible.

In [25]:
bf_awards_data = pd.merge(bf_data, hof_awards, on='playerID', how='outer')
bf_awards_data = bf_awards_data.fillna(0)
bf_awards_data.head()

,playerID,G,PO,E,RF,FP,H,2B,3B,HR,...,SB,SO,BB,BA,SLG,OBP,OPS,MVP,GG,RoY
0,aaronha01,3020.0,7436.0,144.0,2.60,0.982,3771.0,624.0,98.0,755.0,...,240.0,1383.0,1402.0,0.305,0.555,0.421,0.976,1.0,3.0,0.0
1,adamsbo03,1066.0,1748.0,158.0,3.76,0.962,1082.0,188.0,49.0,37.0,...,67.0,447.0,414.0,0.269,0.368,0.376,0.744,0.0,0.0,0.0
2,adamssp01,1384.0,2287.0,230.0,4.36,0.963,1588.0,249.0,48.0,9.0,...,154.0,223.0,453.0,0.286,0.353,0.372,0.725,0.0,0.0,0.0
3,ageeto01,1073.0,2371.0,61.0,2.26,0.975,999.0,170.0,27.0,130.0,...,167.0,918.0,342.0,0.255,0.412,0.351,0.763,0.0,2.0,1.0
4,allenbe01,1023.0,2031.0,100.0,4.28,0.978,815.0,140.0,21.0,73.0,...,13.0,424.0,370.0,0.239,0.357,0.350,0.707,0.0,0.0,0.0


Finally, the binary classification variable __inducted_bin__ is added to the dataframe. While reviewing the data, I noticed that 44 pitchers had made it into the final dataset; these pitchers had won gold gloves or MVP's at some point in their career and had thus snuck past the filters. This was skewing the data distribution (since all pitchers were removed from the hitting and fielding data, their rows were empty save for the awards). Thankfully they made up the final 44 rows of the dataframe, so it was easy to simply drop them by index. This leaves us with a final 774 x 22 data matrix (22 features are tracked for a total of 774 players). This higher-dimensional data should hopefully add much more accuracy to the model than in my previous iteration, which only tracked 5 features.

In [41]:
#final dataframe:
data_full = pd.merge(bf_awards_data, hof_2, on='playerID')
#print(len(data_full)) = 818
#print(len(data_full[data_full.G == 0.0])) = 44

numlist = list(range(774,818))
data_full = data_full.drop(numlist)
#print(len(data_full)) = 774

774


In [42]:
data_full.to_csv('player_data.csv')

The application of statistical and classification methods to this data will be documented in a separate notebook. The primary objective is to design one (or several) models which will be able to predict, within a reasonable degree of accuracy, whether a player can be expected to recieve a Hall of Fame induction given their career performace as reflected in the above metrics. Additionally, the analysis will also reveal the degree to which each of the tracked features contribute to a players' likelihood of induction. Finally, through the application of more advanced algorithms, clusters and inherent structures in the data may be revealed which can aid in the classification of players.